# Training Models


Imports


In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import (
    classification_report,
    precision_score,
    recall_score,
    accuracy_score,
    f1_score,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Dropout
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv("../Data/cleaned_dataset.csv")

In [3]:
X = df.iloc[:, 2:-1]
y = df.iloc[:, -1]

In [4]:
sm = SMOTE()

In [5]:
X_res, y_res = sm.fit_resample(X, y)

In [6]:
df = pd.concat([X_res, y_res], axis=1)

In [7]:
train_df, test_df = train_test_split(df, test_size=0.2)

Get the training features and label


In [8]:
X_train = train_df.iloc[:, :-1].to_numpy()
y_train = train_df.iloc[:, -1].to_numpy()

Get the testing features and label


In [9]:
X_test = test_df.iloc[:, :-1].to_numpy()
y_test = test_df.iloc[:, -1].to_numpy()

Initialize a 5-fold cross-validation object


In [10]:
KF = KFold(n_splits=5)

## KNN


In [11]:
KNN = KNeighborsClassifier()

In [12]:
param_grid = {"n_neighbors": list(range(1, int(np.sqrt(len(X_train))), 2))}

In [13]:
grid_search = GridSearchCV(estimator=KNN, param_grid=param_grid, cv=KF, verbose=2)

In [14]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 61 candidates, totalling 305 fits
[CV] END ......................................n_neighbors=1; total time=   0.3s
[CV] END ......................................n_neighbors=1; total time=   0.2s
[CV] END ......................................n_neighbors=1; total time=   0.2s
[CV] END ......................................n_neighbors=1; total time=   0.2s
[CV] END ......................................n_neighbors=1; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.3s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=3; total time=   0.2s
[CV] END ......................................n_neighbors=5; total time=   0.2s
[CV] END ......................................

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21,
                                         23, 25, 27, 29, 31, 33, 35, 37, 39, 41,
                                         43, 45, 47, 49, 51, 53, 55, 57, 59, ...]},
             verbose=2)

In [15]:
best_params = grid_search.best_params_
print(best_params)

{'n_neighbors': 1}


In [16]:
KNN_best = KNeighborsClassifier(**best_params)

In [17]:
KNN_best.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [18]:
predictions = KNN_best.predict(X_test)

In [19]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.92      0.65      0.76      1838
           1       0.74      0.95      0.83      1940

    accuracy                           0.80      3778
   macro avg       0.83      0.80      0.80      3778
weighted avg       0.83      0.80      0.80      3778



## Naive Bayes


In [20]:
NB = GaussianNB()

In [21]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

In [22]:
fold = 1
for train, test in KF.split(X_train, y_train):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    NB.fit(X_train[train], y_train[train])

    # Predict on the test set
    predictions = NB.predict(X_train[test])

    # Evaluate the model
    precision = precision_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_train[test], y_pred=predictions)
    f1 = f1_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

##### FOLD: 1 #####
Precision = 0.7379971615828834
Recall = 0.6340172071475844
Accuracy = 0.6340172071475844
F1 score = 0.5928353621745078

##### FOLD: 2 #####
Precision = 0.7514966659304114
Recall = 0.6641297154202515
Accuracy = 0.6641297154202515
F1 score = 0.6284314606411487

##### FOLD: 3 #####
Precision = 0.7368702994964665
Recall = 0.6376571806750496
Accuracy = 0.6376571806750496
F1 score = 0.5940031651153616

##### FOLD: 4 #####
Precision = 0.74464651904674
Recall = 0.6244209133024488
Accuracy = 0.6244209133024488
F1 score = 0.5783864406260321

##### FOLD: 5 #####
Precision = 0.7265919827606105
Recall = 0.6313699536730641
Accuracy = 0.6313699536730641
F1 score = 0.5915649942854433



In [23]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Mean Scores:
Mean Precision = 0.7395205257634224
Mean Recall = 0.6383189940436796
Mean Accuracy = 0.6383189940436796
Mean F1 score = 0.5970442845684987


In [24]:
predictions = NB.predict(X_test)

In [25]:
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.89      0.32      0.47      1838
           1       0.60      0.96      0.74      1940

    accuracy                           0.65      3778
   macro avg       0.74      0.64      0.60      3778
weighted avg       0.74      0.65      0.61      3778



## CART


In [26]:
Cart = DecisionTreeClassifier()

In [27]:
param_grid = {
    "max_depth": [None, 10, 20, 30],
    "class_weight": [
        None,
        {0: 1, 1: 2},
        {0: 1, 1: 3},
        {0: 1.5, 1: 2.5},
        {0: 1.5, 1: 3},
    ],
    "ccp_alpha": [0, 1, 0.01, 0.001],
    "min_samples_split": [2, 3, 4],
}

In [28]:
grid_search = GridSearchCV(estimator=Cart, param_grid=param_grid, cv=KF, verbose=2)

In [29]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 240 candidates, totalling 1200 fits
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.5s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=2; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=3; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=3; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=3; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_samples_split=3; total time=   0.4s
[CV] END ccp_alpha=0, class_weight=None, max_depth=None, min_sa

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(),
             param_grid={'ccp_alpha': [0, 1, 0.01, 0.001],
                         'class_weight': [None, {0: 1, 1: 2}, {0: 1, 1: 3},
                                          {0: 1.5, 1: 2.5}, {0: 1.5, 1: 3}],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_split': [2, 3, 4]},
             verbose=2)

In [30]:
best_params = grid_search.best_params_
print(best_params)

{'ccp_alpha': 0, 'class_weight': None, 'max_depth': 10, 'min_samples_split': 3}


In [31]:
Cart_best = DecisionTreeClassifier(max_depth=10, min_samples_split=3)

In [32]:
Cart_best.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=35)

In [33]:
predictions = Cart_best.predict(X_test)

In [34]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.81      0.77      0.79      1838
           1       0.79      0.83      0.81      1940

    accuracy                           0.80      3778
   macro avg       0.80      0.80      0.80      3778
weighted avg       0.80      0.80      0.80      3778



## Random Forest


In [35]:
RF = RandomForestClassifier(n_jobs=5)

In [36]:
param_grid = {
    "n_estimators": [50, 100, 200, 300, 400],
    "max_depth": [None, 10, 20, 30],
    "class_weight": [
        "balanced",
        "balanced_subsample",
        {0: 1, 1: 2},
        {0: 1, 1: 3},
        {0: 1.5, 1: 2.5},
        {0: 1.5, 1: 3},
    ],
    "ccp_alpha": [1, 0.01, 0.001],
}

In [37]:
grid_search = GridSearchCV(estimator=RF, param_grid=param_grid, cv=KF, verbose=2)

In [38]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None, n_estimators=50; total time=   0.4s
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None, n_estimators=50; total time=   0.3s
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None, n_estimators=50; total time=   0.3s
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None, n_estimators=50; total time=   0.3s
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None, n_estimators=50; total time=   0.3s
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None, n_estimators=100; total time=   0.6s
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None, n_estimators=100; total time=   0.6s
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None, n_estimators=100; total time=   0.6s
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None, n_estimators=100; total time=   0.6s
[CV] END ccp_alpha=1, class_weight=balanced, max_depth=None

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(n_jobs=5),
             param_grid={'ccp_alpha': [1, 0.01, 0.001],
                         'class_weight': ['balanced', 'balanced_subsample',
                                          {0: 1, 1: 2}, {0: 1, 1: 3},
                                          {0: 1.5, 1: 2.5}, {0: 1.5, 1: 3}],
                         'max_depth': [None, 10, 20, 30],
                         'n_estimators': [50, 100, 200, 300, 400]},
             verbose=2)

In [39]:
best_params = grid_search.best_params_
print(best_params)

{'ccp_alpha': 0.001, 'class_weight': 'balanced', 'max_depth': None, 'n_estimators': 400}


In [40]:
RF_best = RandomForestClassifier(**best_params)

In [41]:
RF_best.fit(X_train, y_train)

RandomForestClassifier(ccp_alpha=0.001, class_weight='balanced',
                       n_estimators=400)

In [42]:
predictions = RF_best.predict(X_test)

In [43]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.80      0.79      0.79      1838
           1       0.80      0.81      0.81      1940

    accuracy                           0.80      3778
   macro avg       0.80      0.80      0.80      3778
weighted avg       0.80      0.80      0.80      3778



## Logistic Regression


In [44]:
LR = LogisticRegression(n_jobs=5)

In [45]:
param_grid = {
    "tol": [0.001, 0.0001, 0.00001],
    "solver": ["lbfgs", "liblinear"],
    "max_iter": [2000, 4000, 6000],
    "class_weight": [
        {0: 1, 1: 2},
        {0: 1, 1: 3},
        {0: 1.5, 1: 2.5},
        {0: 1.5, 1: 3},
    ],
}

In [46]:
grid_search = GridSearchCV(estimator=LR, param_grid=param_grid, cv=KF, verbose=2)

In [47]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END class_weight={0: 1, 1: 2}, max_iter=2000, solver=lbfgs, tol=0.001; total time=   6.3s


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV] END class_weight={0: 1, 1: 2}, max_iter=2000, solver=lbfgs, tol=0.001; total time=   6.0s


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1216: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 5.
  warnings.warn(


[CV] END class_weight={0: 1.5, 1: 3}, max_iter=4000, solver=liblinear, tol=0.0001; total time=  12.1s


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1216: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 5.
  warnings.warn(


[CV] END class_weight={0: 1.5, 1: 3}, max_iter=4000, solver=liblinear, tol=0.0001; total time=  15.7s


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1216: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 5.
  warnings.warn(


In [ ]:
best_params = grid_search.best_params_
print(best_params)

{'class_weight': {0: 1.5, 1: 2.5}, 'max_iter': 2000, 'solver': 'liblinear', 'tol': 0.001}


In [ ]:
LR_best = LogisticRegression(**best_params)

In [ ]:
LR_best.fit(X_train, y_train)

LogisticRegression(class_weight={0: 1.5, 1: 2.5}, max_iter=2000,
                   solver='liblinear', tol=0.001)

In [ ]:
predictions = LR_best.predict(X_test)

In [ ]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.90      0.77      0.83      1907
           1       0.59      0.80      0.68       798

    accuracy                           0.77      2705
   macro avg       0.74      0.78      0.75      2705
weighted avg       0.81      0.77      0.78      2705



## SVM


In [ ]:
SVM = SVC()

In [ ]:
param_grid = {
    "tol": [0.001, 0.0001, 0.00001],
    "kernel": ["poly", "rbf", "sigmoid"],
    "class_weight": [
        {0: 1, 1: 2},
        {0: 1, 1: 2.5},
        {0: 1.5, 1: 2.5},
        {0: 1, 1: 3},
    ],
}

In [ ]:
grid_search = GridSearchCV(estimator=SVM, param_grid=param_grid, cv=KF, verbose=2)

In [ ]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END ..class_weight={0: 1, 1: 2}, kernel=poly, tol=0.001; total time=  24.0s
[CV] END ..class_weight={0: 1, 1: 2}, kernel=poly, tol=0.001; total time=  24.0s
[CV] END ..class_weight={0: 1, 1: 2}, kernel=poly, tol=0.001; total time=  23.1s
[CV] END ..class_weight={0: 1, 1: 2}, kernel=poly, tol=0.001; total time=  22.2s
[CV] END ..class_weight={0: 1, 1: 2}, kernel=poly, tol=0.001; total time=  22.8s
[CV] END .class_weight={0: 1, 1: 2}, kernel=poly, tol=0.0001; total time=  23.2s
[CV] END .class_weight={0: 1, 1: 2}, kernel=poly, tol=0.0001; total time=  22.9s
[CV] END .class_weight={0: 1, 1: 2}, kernel=poly, tol=0.0001; total time=  23.3s
[CV] END .class_weight={0: 1, 1: 2}, kernel=poly, tol=0.0001; total time=  23.0s
[CV] END .class_weight={0: 1, 1: 2}, kernel=poly, tol=0.0001; total time=  23.3s
[CV] END ..class_weight={0: 1, 1: 2}, kernel=poly, tol=1e-05; total time=  23.4s
[CV] END ..class_weight={0: 1, 1: 2}, kernel=po

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SVC(),
             param_grid={'class_weight': [{0: 1, 1: 2}, {0: 1, 1: 2.5},
                                          {0: 1.5, 1: 2.5}, {0: 1, 1: 3}],
                         'kernel': ['poly', 'rbf', 'sigmoid'],
                         'tol': [0.001, 0.0001, 1e-05]},
             verbose=2)

In [ ]:
best_params = grid_search.best_params_
print(best_params)

{'class_weight': {0: 1.5, 1: 2.5}, 'kernel': 'poly', 'tol': 0.001}


In [ ]:
SVM_best = SVC(**best_params)

In [ ]:
SVM_best.fit(X_train, y_train)

SVC(class_weight={0: 1.5, 1: 2.5}, kernel='poly')

In [ ]:
predictions = SVM_best.predict(X_test)

In [ ]:
print(classification_report(y_true=y_test, y_pred=predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.70      0.99      0.82      1907
           1       0.06      0.00      0.00       798

    accuracy                           0.70      2705
   macro avg       0.38      0.50      0.41      2705
weighted avg       0.51      0.70      0.58      2705



## Neural Network


In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience=10)

In [ ]:
nn = Sequential()
nn.add(Dense(128, activation="relu"))
nn.add(Dropout(0.2))
nn.add(Dense(1, activation="sigmoid"))

nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
precisions = []
recalls = []
accuracies = []
f1_scores = []

In [ ]:
fold = 1
for train, test in KF.split(X_train, y_train):
    print(f"##### FOLD: {fold} #####")

    # Fit the model
    nn.fit(
        X_train[train],
        y_train[train],
        epochs=200,
        batch_size=32,
        validation_data=(X_train[test], y_train[test]),
        callbacks=[early_stopping],
        verbose=0,
    )

    # Predict on the test set
    predictions = NB.predict(X_train[test])

    # Evaluate the model
    precision = precision_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    recall = recall_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )
    accuracy = accuracy_score(y_true=y_train[test], y_pred=predictions)
    f1 = f1_score(
        y_true=y_train[test],
        y_pred=predictions,
        zero_division=0,
        average="weighted",
    )

    # Store the result
    precisions.append(precision)
    recalls.append(recall)
    accuracies.append(accuracy)
    f1_scores.append(f1)

    # Print the scores for each fold
    print(f"Precision = {precision}")
    print(f"Recall = {recall}")
    print(f"Accuracy = {accuracy}")
    print(f"F1 score = {f1}\n")

    fold += 1

##### FOLD: 1 #####
Precision = 0.7785545430365338
Recall = 0.4967652495378928
Accuracy = 0.4967652495378928
F1 score = 0.47675461320411033

##### FOLD: 2 #####
Precision = 0.780558166314544
Recall = 0.511090573012939
Accuracy = 0.511090573012939
F1 score = 0.49950738308334497

##### FOLD: 3 #####
Precision = 0.7754098685752371
Recall = 0.505083179297597
Accuracy = 0.505083179297597
F1 score = 0.48852320246640313

##### FOLD: 4 #####
Precision = 0.7612125418251275
Recall = 0.5242718446601942
Accuracy = 0.5242718446601942
F1 score = 0.5094383749575703

##### FOLD: 5 #####
Precision = 0.7639858675845088
Recall = 0.5122515025427646
Accuracy = 0.5122515025427646
F1 score = 0.4950346893836649



In [ ]:
print("Mean Scores:")
print(f"Mean Precision = {np.mean(precisions)}")
print(f"Mean Recall = {np.mean(recalls)}")
print(f"Mean Accuracy = {np.mean(accuracies)}")
print(f"Mean F1 score = {np.mean(f1_scores)}")

Mean Scores:
Mean Precision = 0.7719441974671902
Mean Recall = 0.5098924698102776
Mean Accuracy = 0.5098924698102776
Mean F1 score = 0.49385165261901864


In [ ]:
y_pred = nn.predict(X_test)

predictions = []
for pred in y_pred:
    if pred > 0.5:
        predictions.append(1)
    else:
        predictions.append(0)

85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step


In [ ]:
print(classification_report(y_test, predictions, zero_division=0))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      1907
           1       0.69      0.65      0.67       798

    accuracy                           0.81      2705
   macro avg       0.77      0.76      0.77      2705
weighted avg       0.81      0.81      0.81      2705

